In [20]:
import nltk
from nltk.corpus import treebank
import json
import math
from collections import defaultdict, Counter

# Якщо це вперше, потрібно завантажити корпус:
nltk.download('treebank')
nltk.download('universal_tagset')

# Завантажимо корпус з Universal POS тегами
tagged_sentences = list(treebank.tagged_sents(tagset='universal'))

# Розділимо на train і test
split_index = int(0.8 * len(tagged_sentences))
train_data = tagged_sentences[:split_index]
test_data = tagged_sentences[split_index:]

print(f'Кількість тренувальних речень: {len(train_data)}')
print(f'Кількість тестових речень: {len(test_data)}')


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


Кількість тренувальних речень: 3131
Кількість тестових речень: 783


In [21]:
# Ініціалізуємо словники для підрахунку
transition_counts = defaultdict(Counter)
emission_counts = defaultdict(Counter)
tag_counts = Counter()

# Спеціальний стартовий тег
START = "<START>"

for sentence in train_data:
    prev_tag = START
    for word, tag in sentence:
        # Підрахунок переходів тегів
        transition_counts[prev_tag][tag] += 1
        # Підрахунок емісій (словосполучення слово-тег)
        emission_counts[tag][word.lower()] += 1
        # Підрахунок загальної кількості тегів
        tag_counts[tag] += 1
        prev_tag = tag
    # Для кінця речення можна не додавати перехід, або додати перехід в <END>, якщо треба

print("Приклад переходів з тегу 'NOUN':", transition_counts['NOUN'])
print("Приклад емісій для тегу 'NOUN':", emission_counts['NOUN'])


Приклад переходів з тегу 'NOUN': Counter({'NOUN': 6025, '.': 5652, 'ADP': 4033, 'VERB': 3347, 'CONJ': 977, 'PRT': 962, 'X': 680, 'ADV': 412, 'DET': 285, 'ADJ': 255, 'NUM': 218, 'PRON': 117})
Приклад емісій для тегу 'NOUN': Counter({'%': 328, 'mr.': 318, 'u.s.': 187, 'company': 170, 'market': 150, 'year': 147, 'new': 130, 'president': 128, 'program': 120, 'stock': 118, 'trading': 118, 'years': 103, 'corp.': 85, 'york': 81, 'inc.': 76, 'japan': 72, 'business': 67, 'funds': 66, 'shares': 66, 'government': 65, 'time': 63, 'co.': 61, 'investors': 60, 'department': 60, 'prices': 60, 'money': 58, 'companies': 58, 'mrs.': 58, 'share': 57, 'bank': 56, 'board': 55, 'price': 55, 'group': 53, 'index': 53, 'securities': 52, 'people': 52, 'stocks': 52, 'sales': 51, 'banks': 51, 'days': 48, 'october': 48, 'research': 47, 'house': 47, 'investment': 47, 'rates': 46, 'months': 46, 'exchange': 46, 'markets': 46, 'yesterday': 45, 'week': 44, 'congress': 43, 'futures': 43, 'trade': 42, 'country': 42, 'inte

In [22]:
# Отримаємо список усіх тегів (станів)
tags = list(tag_counts.keys())

# Підрахунок ймовірностей переходів P(t_i | t_{i-1})
transition_probs = defaultdict(dict)
for prev_tag in transition_counts:
    total = sum(transition_counts[prev_tag].values()) + len(tags)  # add-one згладжування
    for tag in tags:
        count = transition_counts[prev_tag][tag] + 1  # +1 для add-one
        transition_probs[prev_tag][tag] = math.log(count / total)  # Логарифм для стабільності

# Підрахунок ймовірностей емісій P(w | t)
emission_probs = defaultdict(dict)
for tag in emission_counts:
    total = sum(emission_counts[tag].values()) + len(emission_counts[tag]) + 1  # add-one + невідомі слова
    vocab_size = len(emission_counts[tag])
    for word in emission_counts[tag]:
        count = emission_counts[tag][word] + 1
        emission_probs[tag][word] = math.log(count / total)

# Додамо ймовірність для невідомих слів (UNK)
for tag in tags:
    emission_probs[tag]['<UNK>'] = math.log(1 / (sum(emission_counts[tag].values()) + vocab_size + 1))

print("Приклад ймовірностей переходу з 'NOUN' до 'VERB':", transition_probs['NOUN'].get('VERB'))
print("Приклад ймовірностей емісії для 'dog' в тегу 'NOUN':", emission_probs['NOUN'].get('dog', emission_probs['NOUN']['<UNK>']))


Приклад ймовірностей переходу з 'NOUN' до 'VERB': -1.926043515614522
Приклад ймовірностей емісії для 'dog' в тегу 'NOUN': -10.043423392833645


In [23]:
def viterbi(sentence, transition_probs, emission_probs, tags):
    V = [{}]
    path = {}

    START = "<START>"

    # Ініціалізація першого кроку
    for tag in tags:
        trans_p = transition_probs[START].get(tag, float('-inf'))
        emis_p = emission_probs[tag].get(sentence[0].lower(), emission_probs[tag]['<UNK>'])
        V[0][tag] = trans_p + emis_p
        path[tag] = [tag]

    # Ітеруємо по словах речення
    for t in range(1, len(sentence)):
        V.append({})
        new_path = {}

        for tag in tags:
            (prob, state) = max(
                (V[t-1][prev_tag] + transition_probs[prev_tag].get(tag, float('-inf')) + emission_probs[tag].get(sentence[t].lower(), emission_probs[tag]['<UNK>']), prev_tag)
                for prev_tag in tags
            )
            V[t][tag] = prob
            new_path[tag] = path[state] + [tag]

        path = new_path

    # Знаходимо найкращий фінальний стан
    n = len(sentence) - 1
    (prob, state) = max((V[n][tag], tag) for tag in tags)
    return path[state]


In [24]:
def accuracy(test_data, transition_probs, emission_probs, tags):
    total_tags = 0
    correct_tags = 0

    for sentence in test_data:
        words = [w for w, t in sentence]
        true_tags = [t for w, t in sentence]
        pred_tags = viterbi(words, transition_probs, emission_probs, tags)

        total_tags += len(true_tags)
        for true_tag, pred_tag in zip(true_tags, pred_tags):
            if true_tag == pred_tag:
                correct_tags += 1

    return correct_tags / total_tags

acc = accuracy(test_data, transition_probs, emission_probs, tags)
print(f'Точність власної реалізації HMM POS-тегування: {acc:.4f}')


Точність власної реалізації HMM POS-тегування: 0.8591


In [25]:
from nltk.tag import UnigramTagger, DefaultTagger

# Використовуємо UnigramTagger на тренувальних даних
unigram_tagger = UnigramTagger(train_data, backoff=DefaultTagger('NOUN'))

def nltk_accuracy(tagger, test_data):
    total = 0
    correct = 0
    for sent in test_data:
        words = [w for w, t in sent]
        gold_tags = [t for w, t in sent]
        pred_tags = [tag for (word, tag) in tagger.tag(words)]
        total += len(gold_tags)
        correct += sum(1 for gt, pt in zip(gold_tags, pred_tags) if gt == pt)
    return correct / total

acc_nltk = nltk_accuracy(unigram_tagger, test_data)
print(f'Точність NLTK UnigramTagger: {acc_nltk:.4f}')


Точність NLTK UnigramTagger: 0.9337


In [26]:
# Для збереження словників з логарифмами конвертуємо в звичайні float
def convert_to_serializable(d):
    if isinstance(d, dict) or isinstance(d, defaultdict):
        return {k: convert_to_serializable(v) for k, v in d.items()}
    elif isinstance(d, float):
        return float(d)
    else:
        return d

model = {
    'transition_probs': convert_to_serializable(transition_probs),
    'emission_probs': convert_to_serializable(emission_probs),
    'tags': tags
}

with open('hmm_pos_model.json', 'w', encoding='utf-8') as f:
    json.dump(model, f, ensure_ascii=False, indent=2)


In [27]:
import json

# Шлях до файлу моделі (завантаж у свій Colab)
model_path = 'hmm_pos_model.json'

# Завантаження JSON-моделі
with open(model_path, 'r', encoding='utf-8') as f:
    model = json.load(f)

# Виведення основної інформації
print("Теги (tags):")
print(model.get('tags', []))
print()

print("Кількість тегів:", len(model.get('tags', [])))
print()

print("Приклади transition_probs:")
for prev_tag, transitions in list(model.get('transition_probs', {}).items())[:3]:
    print(f"{prev_tag} -> {transitions}")

print()

print("Приклади emission_probs:")
for tag, emissions in list(model.get('emission_probs', {}).items())[:3]:
    print(f"{tag}:")
    # Покажемо перші 5 слів із ймовірностями
    for word, prob in list(emissions.items())[:5]:
        print(f"  {word} : {prob}")
    print()

# Додатково: перевірити чи всі слова мають тег PRON (як у тебе було)
all_tags = model.get('tags', [])
emission_probs = model.get('emission_probs', {})

pron_words = emission_probs.get('PRON', {}).keys() if 'PRON' in emission_probs else []

print("Слова, які мають тег PRON (перші 10):", list(pron_words)[:10])


Теги (tags):
['NOUN', '.', 'NUM', 'ADJ', 'VERB', 'DET', 'ADP', 'CONJ', 'X', 'ADV', 'PRT', 'PRON']

Кількість тегів: 12

Приклади transition_probs:
<START> -> {'NOUN': -1.3399768361204978, '.': -2.369353269458886, 'NUM': -4.757096170793239, 'ADJ': -3.2166511298460896, 'VERB': -4.651735655135412, 'DET': -1.454424008183052, 'ADP': -2.01506211687543, 'CONJ': -2.9112694802949077, 'X': -3.6221162379542537, 'ADV': -2.8996414422997887, 'PRT': -6.261173567569513, 'PRON': -2.5077555923180053}
NOUN -> {'NOUN': -1.3383232275336412, '.': -1.402220290548597, 'NUM': -4.653090217407387, 'ADJ': -4.496984502744326, 'VERB': -1.926043515614522, 'DET': -4.3861701364040355, 'ADP': -1.7396482287097308, 'CONJ': -3.1566522771890706, 'X': -3.5185996410743754, 'ADV': -4.018714354262855, 'PRT': -3.172108535425762, 'PRON': -5.271477322758223}
. -> {'NOUN': -1.6443168361370102, '.': -2.22130304141244, 'NUM': -2.270093205581872, 'ADJ': -3.1239783712685973, 'VERB': -2.055470703999596, 'DET': -1.9588277829940055, 'ADP